In [29]:
import os
from google.colab import files
import torchvision
import glob
from torchvision import transforms
import numpy as np
import torch

#Batch preparing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
path = '/content/drive/MyDrive/Colab_Notebooks/datasets/stm-data'
train_bad_dir = path + '/train/bad'
train_good_dir = path + '/train/good'
test_bad_dir = path + '/test/bad'
test_good_dir = path + '/test/good'

train_bad_names = glob.glob(train_bad_dir + '/*.npy')
train_good_names = glob.glob(train_good_dir + '/*.npy')
test_bad_names = glob.glob(test_bad_dir + '/*.npy')
test_good_names = glob.glob(test_good_dir + '/*.npy')

In [44]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, names, y, transform = None):
        self.names = names
        self.y = y
        if transform is None:
            self.should_transform = False
        else:
            self.transform = transform
            self.should_transform = True

    def __len__(self):
        return len(self.names)

    def __getitem__(self,idx):
        mas = np.load(self.names[idx])
        if self.should_transform:
            mas_transformed = self.transform(mas)
        else:
            mas_transformed = mas
        return mas_transformed, self.y

In [56]:
transform = torchvision.transforms.Compose([                                    
    transforms.ToTensor()                                       
    ])

train_bad_dataset = MyDataset(names=train_bad_names, y='bad', transform=transform)
train_good_dataset = MyDataset(names=train_good_names, y='good', transform=transform)
train_dataset = train_bad_dataset + train_good_dataset

test_bad_dataset = MyDataset(names=test_bad_names, y='bad', transform=transform)
test_good_dataset = MyDataset(names=test_good_names, y='good', transform=transform)
test_dataset = test_bad_dataset + test_good_dataset

In [57]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=True)